# _utils.misc

> TODO fill in description

In [ ]:
#| default_exp _utils.misc

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
import re, keyword
import ast
from pathlib import Path
import subprocess
import os, sys
import importlib.util
import socket
import random
from abc import ABC, abstractmethod

import fbdev

In [ ]:
#|export
def is_valid_name(name: str) -> bool:
    for name_part in name.split('.'):
        if keyword.iskeyword(name_part):
            return False
        valid_identifier_pattern = r'^[A-Za-z_][A-Za-z0-9_]*$'
        if not re.match(valid_identifier_pattern, name_part): return False
    return True

In [ ]:
# Example usage:
assert is_valid_name("my_var")
assert not is_valid_name("2var")
assert not is_valid_name("def")
assert is_valid_name("my.var")
assert not is_valid_name("my.1")
assert not is_valid_name("my.var!")

In [ ]:
#|hide
show_doc(fbdev._utils.is_mutually_exclusive)

---

### is_mutually_exclusive

>      is_mutually_exclusive (*args, at_least_one:bool=False)

In [ ]:
#|export
def is_mutually_exclusive(*args, at_least_one:bool=False):
    num_not_None = sum(map(lambda a: not a is None, args))
    
    if at_least_one: return num_not_None == 1
    else: return num_not_None <= 1

In [ ]:
assert is_mutually_exclusive(1)
assert is_mutually_exclusive(1, None)
assert not is_mutually_exclusive(1, None, 2)
assert not is_mutually_exclusive(None, None, None, at_least_one=True)
assert is_mutually_exclusive(None, 1, None, at_least_one=True)

In [ ]:
#|hide
show_doc(fbdev._utils.is_in_event_loop)

---

### is_in_event_loop

>      is_in_event_loop ()

In [ ]:
#|export
def is_in_event_loop():
    try:
        asyncio.get_running_loop()
        return True
    except RuntimeError:
        return False

In [ ]:
#|hide
show_doc(fbdev._utils.get_git_root_directory)

---

### get_git_root_directory

>      get_git_root_directory ()

In [ ]:
#|export
def get_git_root_directory():
    try:
        # Run 'git rev-parse --show-toplevel' command
        git_root = subprocess.check_output(['git', 'rev-parse', '--show-toplevel'], stderr=subprocess.STDOUT).decode('utf-8').strip()
        return git_root
    except subprocess.CalledProcessError:
        # If the command fails, it means we're not in a git repository
        return None

In [ ]:
get_git_root_directory()

'/Users/lukastk/proj_dev/fbdev'

In [ ]:
cwd = os.getcwd()
os.chdir(os.path.expanduser("~"))
print(get_git_root_directory() is None)
os.chdir(cwd)

True


In [ ]:
#|hide
show_doc(fbdev._utils.root_dir)

---

### root_dir

>      root_dir (module)

In [ ]:
#|export
def root_dir(module):
    return Path(module.__file__).parent

In [ ]:
root_dir(fbdev)

Path('/Users/lukastk/proj_dev/fbdev/fbdev')

In [ ]:
#|hide
show_doc(fbdev._utils.extract_top_level_docstring)

---

### extract_top_level_docstring

>      extract_top_level_docstring (file_path)

In [ ]:
#|export
def extract_top_level_docstring(file_path):
    with open(file_path, "r") as file:
        file_content = file.read()
    
    # Parse the file content using AST
    parsed_ast = ast.parse(file_content)
    
    # Retrieve the docstring of the module
    docstring = ast.get_docstring(parsed_ast)
    
    return docstring

In [ ]:
get_git_root_directory()

'/Users/lukastk/proj_dev/fbdev'

In [ ]:
# Example usage
file_path = Path(get_git_root_directory()) / "fbdev" / "exceptions.py"
docstring = extract_top_level_docstring(file_path)
if docstring:
    print("Top-level docstring:", docstring)
else:
    print("No top-level docstring found.")

Top-level docstring: TODO fill in description


In [ ]:
#|hide
show_doc(fbdev._utils.find_module_root)

---

### find_module_root

>      find_module_root (path)

In [ ]:
#|export
def find_module_root(path):
    path = Path(path)
    path = path if path.is_dir() else path.parent
    is_module = '__init__.py' in [p.parts[-1] for p in path.glob('*')]
    if not is_module: return None
    else:
        parent_module = find_module_root(path.parent)
        if parent_module is None: return path
        else: return parent_module

In [ ]:
find_module_root(Path(fbdev.__path__[0]) / 'utils.py')

Path('/Users/lukastk/proj_dev/fbdev/fbdev')

In [ ]:
#|hide
show_doc(fbdev._utils.get_module_path_hierarchy)

---

### get_module_path_hierarchy

>      get_module_path_hierarchy (path)

In [ ]:
#|export        
def get_module_path_hierarchy(path):
    hierarchy = []
    __get_module_path_hierarchy(path, hierarchy)
    return hierarchy

In [ ]:
#|exporti
def __get_module_path_hierarchy(path, hierarchy):
    path = Path(path)
    if not path.exists(): raise FileNotFoundError(f"No file or directory found at: {path}")
    if path.is_file():
        if path.suffix != '.py': raise ValueError(f"File '{path}' is not a python file.")
        is_in_module = '__init__.py' in [p.parts[-1] for p in path.parent.glob('*')]
        if is_in_module:
            module_name = path.stem
            hierarchy.append((module_name, path))
            __get_module_path_hierarchy(path.parent, hierarchy)
    else:
        is_module = '__init__.py' in [p.parts[-1] for p in path.glob('*')]
        if is_module:
            module_name = path.stem
            hierarchy.append((module_name, path))
            __get_module_path_hierarchy(path.parent, hierarchy)

In [ ]:
get_module_path_hierarchy(Path(fbdev.__path__[0]) / '_utils.py')

[('_utils', Path('/Users/lukastk/proj_dev/fbdev/fbdev/_utils.py')),
 ('fbdev', Path('/Users/lukastk/proj_dev/fbdev/fbdev'))]

In [ ]:
#|hide
show_doc(fbdev._utils.get_function_from_py_file)

---

### get_function_from_py_file

>      get_function_from_py_file (file_path, func_name=None, args=[],
>                                 is_async=False)

In [ ]:
#|export
import types

In [ ]:
#|export
def get_function_from_py_file(file_path, func_name=None, args=[], is_async=False):
    file_path = Path(file_path)
    module_path = find_module_root(file_path)
    is_in_module = module_path is not None
    
    # Check if the file exists
    if not file_path.is_file():
        raise ValueError(f"Not a file: {file_path}")
    if not file_path.exists():
        raise FileNotFoundError(f"No file found at: {file_path}")
    
    if func_name is None:
        func_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Read the contents of the file
    with open(file_path, 'r') as file:
        func_body_code = file.read()
        
    if not func_body_code.strip(): func_body_code = 'pass'
    
    # Tabify
    func_body_code = '\n'.join(list(map(lambda line: f"    {line}", func_body_code.split('\n'))))
    func_code = f"{'async ' if is_async else ''}def {func_name}({', '.join(args)}):\n{func_body_code}"
    
    if is_in_module:
        # This all is necessary to allow for relative imports in the code
        sys.path.insert(0, module_path.parent.absolute().as_posix())
        module_hierarchy = get_module_path_hierarchy(file_path)
        module_hierarchy_str = '.'.join([e[0] for e in reversed(module_hierarchy)])
        module_spec = importlib.util.spec_from_file_location(module_hierarchy_str, file_path.absolute().as_posix())
        code_module = importlib.util.module_from_spec(module_spec)
        locals_dict = code_module.__dict__
    else:
        locals_dict = {}
        
    exec(func_code, locals_dict)
    if is_in_module: sys.path.pop(0)
    
    func = locals_dict[func_name]
    # Create a new code object with the correct filename and line number. This will allow for proper displaying of the line number and code during exceptions.
    new_code = types.CodeType(
        func.__code__.co_argcount,
        func.__code__.co_posonlyargcount,
        func.__code__.co_kwonlyargcount,
        func.__code__.co_nlocals,
        func.__code__.co_stacksize,
        func.__code__.co_flags,
        func.__code__.co_code,
        func.__code__.co_consts,
        func.__code__.co_names,
        func.__code__.co_varnames,
        file_path.as_posix(),
        func.__code__.co_name,
        -1, # Line number offset. Not entirely sure why it's -1, but it works.
        func.__code__.co_lnotab,
        func.__code__.co_freevars,
        func.__code__.co_cellvars
    )
    func.__code__ = new_code
    
    return func

In [ ]:
#|hide
show_doc(fbdev._utils.SingletonMeta)

---

### SingletonMeta

>      SingletonMeta (name, bases=(), dct=None)

*type(object) -> the object's type
type(name, bases, dict, **kwds) -> a new type*

In [ ]:
#|export
class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

    def __new__(metacls, name, bases=(), dct=None):
        if dct is None:
            dct = {}
        return super().__new__(metacls, name, bases, dct)

In [ ]:
#|hide
show_doc(fbdev._utils.abstractproperty)

---

### abstractproperty

>      abstractproperty (func)

In [ ]:
#|export
def abstractproperty(func):
    return property(abstractmethod(func))

In [ ]:
#|hide
show_doc(fbdev._utils.find_available_port)

---

### find_available_port

>      find_available_port ()

In [ ]:
#|export
def find_available_port():
    while True:
        port = random.randint(49152, 65535)
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(('localhost', port))
                return port
            except OSError:
                continue

In [ ]:
random.seed(0)
find_available_port()

61775